This script is here just to give me a place to test random code snippets quickly... It probably will never be formatted very well. 

In [1]:
# plotting stuff
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
from wrf import (to_np, getvar, ALL_TIMES, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

# Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
else:
    sys.path.append('/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF
from optwrf import runwrf as rw

/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/pvlib-python/pvlib/wrfcast.py:20: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  'The forecast module algorithms and features are highly experimental. '


In [3]:
# Verbose setting
verbose = True

# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# Open the NetCDF file
ncfile_orig = os.path.join(pvlib_abspath, 'data', 'wrfout_d01.nc')
# In order for cartopy to access the map projection, we MUST read the dataset in with the netCDF4 package. 
ncdata_orig = netCDF4.Dataset(ncfile_orig)
ncfile = os.path.join(pvlib_abspath, 'data', 'wrfout_processed_d01.nc')
ncdata = xr.open_dataset(ncfile)

# Regrid the wrf data
wrfdir = os.path.join(pvlib_abspath, 'data/')
eradir = f'/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/wrf_data/ERA5/'
input_year = 2011
input_month = 2
wrfdata, eradata = rw.wrf_era5_regrid_xesmf(input_year, input_month, wrfdir=wrfdir, eradir=eradir)

# Calculate the error between the WRF simulation and the ERA5 reanalysis
wrfdata = rw.wrf_era5_error(wrfdata, eradata)

Overwrite existing file: bilinear_191x191_97x129.nc 
 You can set reuse_weights=True to save computing time.


/Users/swardy9230/opt/anaconda3/envs/runwrf/lib/python3.7/site-packages/scipy/sparse/_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [5]:
dni_orig = getvar(ncdata_orig, "SWDDNI", timeidx=0)
# Get the latitude and longitude points
lats, lons = latlon_coords(dni_orig)
lats

<xarray.DataArray 'XLAT' (south_north: 191, west_east: 191)>
array([[30.952759, 30.950832, 30.948757, ..., 28.050678, 28.022198,
        27.99358 ],
       [31.060238, 31.058304, 31.056229, ..., 28.153461, 28.124924,
        28.096264],
       [31.16774 , 31.165806, 31.163723, ..., 28.256256, 28.227665,
        28.19897 ],
       ...,
       [51.229244, 51.22667 , 51.223907, ..., 47.38171 , 47.34407 ,
        47.306286],
       [51.33512 , 51.332554, 51.329792, ..., 47.48225 , 47.44457 ,
        47.406715],
       [51.44096 , 51.438396, 51.435627, ..., 47.58274 , 47.54501 ,
        47.507103]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -95.43335 -95.308044 ... -64.11832
    XLAT     (south_north, west_east) float32 30.952759 30.950832 ... 47.507103
    XTIME    float32 0.0
    Time     datetime64[ns] 2011-02-07
Dimensions without coordinates: south_north, west_east

In [6]:
# Get the cartopy mapping object
cart_proj = get_cartopy(dni_orig)
cart_proj

/Users/swardy9230/opt/anaconda3/envs/runwrf/lib/python3.7/site-packages/cartopy/io/__init__.py:260: DownloadWarning: Downloading: http://naciscdn.org/naturalearth/110m/physical/ne_110m_coastline.zip
  warnings.warn('Downloading: {}'.format(url), DownloadWarning)


In [8]:
cp = crs.LambertConformal()
cp